In [1]:
import numpy as np
import scipy
import math
import os
PointConfiguration.set_engine('internal')
print('Okay')

Okay


In [2]:
def exist(pts, latt):
    latt = np.array(latt)
    for i in range(pts.shape[0]):
        if pts[i][0]==latt[0]:
            if pts[i][1]==latt[1]:
                if pts[i][2]==latt[2]:
                    return 1
    return 0

def contain(poly, latt):
    if poly.contains(latt) == 1:
        return 1
    else:
        poly_latt = Polyhedron(vertices = [tuple(latt)])
        vert = next(poly_latt.vertex_generator())
        face_eq = poly.Hrepresentation()
        for eq in face_eq:
            if eq.contains(vert) != 1:
                return 0
        return 1
    return 0

def check_latt(p):
    pts = np.array(p)
    pts_max = int(max(np.amax(pts, axis=0)))+1
    pts_min = int(min(np.amin(pts, axis=0)))-1
    print ('pts_max and pts_min: ', pts_max, pts_min)
    poly = Polyhedron(p)
    pts_new = pts
    for i in range(pts_min, pts_max):
        for j in range(pts_min, pts_max):
            for k in range(pts_min, pts_max):
                latt = [i,j,k]
                if exist(pts, latt) == 1:
                    continue
                if contain(poly, latt) == 1:
                    pts_new = np.append(pts_new, np.array(latt).reshape((1,3)), axis = 0)  
    pts_new = pts_new.tolist()
    return pts_new

def four_cross(v1, v2, v3, v4):
    #Compute cross product of three 4-vectors
    #print ("input vectors: ", v1, v2, v3, v4)
    v = np.zeros((4,))
    counter = 0
    
    for i in range(4):
        mat = [v1[np.arange(len(v1))!=i].tolist(), v2[np.arange(len(v2))!=i].tolist(), v3[np.arange(len(v3))!=i].tolist()]
        mat = matrix(ZZ, mat)
        #print ('matrix: ')
        #print (mat)
        if counter == 1:
            v[i] = -1*mat.det()
            counter = 0
            #print ('neg: ', v[i])
            continue
        elif counter == 0:
            v[i] = mat.det()
            counter = 1
            #print ('pos: ', v[i])
    #print v
    mat = matrix(RR, [v1.tolist(), v2.tolist(), v3.tolist(), v4.tolist()])
    
    if mat.det() < 0:
        #print ('original: ', v)
        v = -1*v
        #print('changed: ', v)
    #print ('vector: ', v)
    return v

def Hilb(triang_list):
    triang = []
    # Add 1 at the end of all verticies
    for tetra in triang_list:
        tetra_new = []
        for vert in tetra:
            vert_new = np.append(np.array(vert),1).tolist()
            tetra_new.append(vert_new)
        triang.append(tetra_new)
    
    triang = np.array(triang)
    # Add 1 to the end of all vectors
    power = np.zeros(shape = triang.shape)
    Hilb = 0
    t = var('t')
    t1 = var('t1')
    t2 = var('t2')
    t3 = var('t3')
    t4 = var('t4')
    for tri in range(triang.shape[0]):
        hilb = 1
        t_prod = 1
        for i in range(4):
            #Multiplying by -1 is optional
            power[tri][i] = -1*four_cross(triang[tri][i], triang[tri][np.remainder(i+1, 4)], triang[tri][np.remainder(i+2, 4)], triang[tri][np.remainder(i+3, 4)])
            t_prod = t1^(int(power[tri][i][0]))*t2^(int(power[tri][i][1]))*t3^(int(power[tri][i][2]))*t4^int((power[tri][i][3]))
            hilb *= (1-t_prod)^(-1)
        #print ('Hilbert: ', hilb)
        Hilb += hilb
#     print ('Hilb: ', Hilb())
    #print (Hilb(t1=t, t2=t, t3=t).series(t4, 3))
    #print ("p-q web: ", power )
    
    
    m = var('m')
    b1 = var('b1')
    b2 = var('b2')
    b3 = var('b3')
    b4 = var('b4')
    Hilb *= m^4
    
    #print ('Hilb: ', str(Hilb(t1 = (m*b1).exp(), t2 = (m*b2).exp(), t3 = (m*b3).exp(), t4 = (m*4).exp())).replace('e', 'E'))
    Series = Hilb(t1 = (m*b1).exp(), t2 = (m*b2).exp(), t3 = (m*b3).exp(), t4 = (m*4).exp()).series(m==0, 1)
    
    Series = Series.truncate()
    
    return Series, triang, power

def idx_to_pts(triang, pts):
    # Input a list of lists of indicies
    # Output a list of lists of points
    triang_new = []
    for i in range(len(triang)):
        triang_new.append([pts[j] for j in triang[i]])
    return triang_new

print('Okay')

Okay


In [3]:
def lift_prism(h1, h2, h3):
    # h1 >= h2 >= h3
    
    triang = []
    power = []
    if h1 == h2 == h3:
        series = 0
        prism = []
        # h1 = h2 = h3
        for h in range(h1):
            # 0 <= h < h1
            prism_1 = [[[1,0,0],[0,0,h],[0,0,h+1],[1,1,h3]]]
            prism_2 = [[[1,1,h3],[0,0,h],[0,0,h+1],[0,1,h2]]]
            prism += prism_1 + prism_2
            
            series_1, triang_1, power_1 = Hilb(prism_1)
            series_2, triang_2, power_2 = Hilb(prism_2)
            series += series_1 + series_2
            triang.append(triang_1)
            triang.append(triang_2)
            power.append(power_1)
            power.append(power_2)
        return prism, series, triang, power
            
    if h2 > h3:
        # h1 >= h2 > h3
        assert((h1 >= h2) and (h2 > h3))
        series = 0
        prism = []
        for h in range(h3):
            # 0 <= h < h3
            prism_1 = [[[1,0,0],[0,0,h],[0,0,h+1],[1,1,h3]]]
            prism_2 = [[[0,1,h2],[0,0,h],[0,0,h+1],[1,1,h3]]]
            prism += prism_1 + prism_2
            
            series_1, triang_1, power_1 = Hilb(prism_1)
            series_2, triang_2, power_2 = Hilb(prism_2)
            series += series_1 + series_2
            triang.append(triang_1)
            triang.append(triang_2)
            power.append(power_1)
            power.append(power_2)

        for h in range(h3,h2):
            # h3 <= h < h2
            prism_1 = [[[1,0,0],[0,0,h],[0,0,h+1],[1,1,h3]]]
            prism_2 = [[[0,1,h2],[0,0,h],[0,0,h+1],[1,1,h3]]]
            prism += prism_1 + prism_2
            
            series_1, triang_1, power_1 = Hilb(prism_1)
            series_2, triang_2, power_2 = Hilb(prism_2)
            series += series_1 + series_2
            triang.append(triang_1)
            triang.append(triang_2)
            power.append(power_1)
            power.append(power_2)

        if h1 > h2:
            # h1 > h2 > h3
            assert(h1 > h2 and h2 > h3)
            for h in range(h2,h1):
                # h2 <= h < h1
                prism_1 = [[[1,0,0],[0,0,h],[0,0,h+1],[1,1,h3]]]
                prism_2 = [[[0,1,h2],[0,0,h],[0,0,h+1],[1,1,h3]]]
                prism += prism_1 + prism_2
                
                series_1, triang_1, power_1 = Hilb(prism_1)
                series_2, triang_2, power_2 = Hilb(prism_2)
                series += series_1 + series_2
                triang.append(triang_1)
                triang.append(triang_2)
                power.append(power_1)
                power.append(power_2)
        return prism, series, triang, power
        
    elif h2 < h3:
        series = 0
        prism = []
        if h1 > h3:
            # h1 > h3 > h2
            assert(h1 > h3 and h3 > h2)
            for h in range(h2):
                # 0 <= h < h2
                prism_1 = [[[1,0,0],[0,0,h],[0,0,h+1],[0,1,h2]]]
                prism += prism_1
                
                series_1, triang_1, power_1 = Hilb(prism_1)
                series += series_1
                triang.append(triang_1)
                power.append(power_1)
            
            # Middle prism:
            prism_1 = [[[1,0,0],[0,1,h2],[0,0,h2],[1,1,h3]]]
            prism += prism_1
            
            series_1, triang_1, power_1 = Hilb(prism_1)
            series += series_1
            triang.append(triang_1)
            power.append(power_1)
            
            for h in range(h2,h3):
                # h2 <= h < h3
                prism_1 = [[[1,0,0],[0,0,h],[0,0,h+1],[1,1,h3]]]
                prism_2 = [[[0,1,h2],[0,0,h],[0,0,h+1],[1,1,h3]]]
                prism += prism_1 + prism_2
                
                series_1, triang_1, power_1 = Hilb(prism_1)
                series_2, triang_2, power_2 = Hilb(prism_2)
                series += series_1 + series_2
                triang.append(triang_1)
                triang.append(triang_2)
                power.append(power_1)
                power.append(power_2)
            return prism, series, triang, power
                
        else:
            # h1 = h3 > h2
            assert(h1 == h3 and h3 > h2)
            for h in range(h2):
                # 0 <= h < h2
                prism_1 = [[[1,0,0],[0,0,h],[0,0,h+1],[0,1,h2]]]
                prism += prism_1
                
                series_1, triang_1, power_1 = Hilb(prism_1)
                series += series_1
                triang.append(triang_1)
                power.append(power_1)
                
            # Middle prism:
            prism_1 = [[[1,0,0],[0,1,h2],[0,0,h1],[1,1,h3]]]
            prism += prism_1
            
            series_1, triang_1, power_1 = Hilb(prism_1)
            series += series_1
            triang.append(triang_1)
            power.append(power_1)
                
            for h in range(h2,h1):
                # h2 <= h < h1
                prism_1 = [[[1,0,0],[0,0,h],[0,0,h+1],[0,1,h2]]]
                prism += prism_1
                
                series_1, triang_1, power_1 = Hilb(prism_1)
                series += series_1
                triang.append(triang_1)
                power.append(power_1)
            return prism, series, triang, power
                
    else:
        assert(h2 == h3)
        assert(h1 > h2)
        # h1 > h2 = h3
        prism = []
        series = 0
        for h in range(h2):
            # 0 <= h < h2
            prism_1 = [[[1,0,0],[0,0,h],[0,0,h+1],[1,1,h3]]]
            prism_2 = [[[0,1,h2],[0,0,h],[0,0,h+1],[1,1,h3]]]
            prism += prism_1 + prism_2

            series_1, triang_1, power_1 = Hilb(prism_1)
            series_2, triang_2, power_2 = Hilb(prism_2)
            series += series_1 + series_2
            triang.append(triang_1)
            triang.append(triang_2)
            power.append(power_1)
            power.append(power_2)
        
        for h in range(h2,h1):
            # h2 <= h < h1
            prism_1 = [[[1,0,0],[0,0,h],[0,0,h+1],[1,1,h3]]]
            prism_2 = [[[0,1,h2],[0,0,h],[0,0,h+1],[1,1,h3]]]
            prism += prism_1 + prism_2

            series_1, triang_1, power_1 = Hilb(prism_1)
            series_2, triang_2, power_2 = Hilb(prism_2)
            series += series_1 + series_2
            triang.append(triang_1)
            triang.append(triang_2)
            power.append(power_1)
            power.append(power_2)
        
    return prism, series, triang, power

print('Okay.')

Okay.


In [13]:
def prism_plot(prism):
    count = 0
    plot = 0
    for p in prism:
        poly = Polyhedron(vertices=p)
        if count == 0:
            count += 1
            plot = poly.plot()
        else:
            plot += poly.plot()
    
#     print(poly_list.vertices())
    return plot

prism,series, triang, power = lift_prism(3,0,0)
print(prism)
prism_plot(prism)

[[[1, 0, 0], [0, 0, 0], [0, 0, 1], [1, 1, 0]], [[0, 1, 0], [0, 0, 0], [0, 0, 1], [1, 1, 0]], [[1, 0, 0], [0, 0, 1], [0, 0, 2], [1, 1, 0]], [[0, 1, 0], [0, 0, 1], [0, 0, 2], [1, 1, 0]], [[1, 0, 0], [0, 0, 2], [0, 0, 3], [1, 1, 0]], [[0, 1, 0], [0, 0, 2], [0, 0, 3], [1, 1, 0]]]


" 
 width="100%"
 height="500"
 style="border: 0;">

In [4]:
# numerical solver
import scipy.optimize
import csv
from scipy.optimize import fsolve

def func(p, *d):
    f1, f2, f3 = d
    return (f1(b1 = p[0], b2 = p[1], b3 = p[2]), f2(b1 = p[0], b2 = p[1], b3 = p[2]), f3(b1 = p[0], b2 = p[1], b3 = p[2]))

def constraint(Series, sol, vol_min_global):
    vol = Series(b1 = sol[0], b2 = sol[1], b3 = sol[2])
    vol = abs(vol)
    if vol <= 1 and vol >= vol_min_global:
        return 1, vol
    
    print 'volume: ', vol, ' is out of bounds.'
    
    return 0, -1
    
def NSolve(Series, d, vol_min_global, bound=[[0,1],[0,1],[0,1]], MAX_COUNT=3):
    vol = -1
    sol = -1
    const = 0
    count = 0
    
    b1_min = bound[0][0]
    b1_max = bound[0][1]
    b2_min = bound[1][0]
    b2_max = bound[1][1]
    b3_min = bound[2][0]
    b3_max = bound[2][1]

    while const == 0:
        if count >= MAX_COUNT:
            return vol,sol
            
        count += 1
        d1_0 = np.random.uniform(low=b1_min, high=b1_max)
        d2_0 = np.random.uniform(low=b2_min, high=b2_max)
        d3_0 = np.random.uniform(low=b3_min, high=b3_max)
        print 'reset starting point: ', d1_0, d2_0, d3_0

        try:
            sol = fsolve(func, x0 = np.array([d1_0, d2_0, d3_0]), args = d)
            print 'solution: ', sol
            print 'guessed vol: ', Series(b1 = sol[0], b2 = sol[1], b3 = sol[2])
        except:
            continue
        
        const, vol = constraint(Series, sol, vol_min_global)

    print 'Done.'

    return vol, sol

def solver(series, b_max, h_max, sol_max=100):
    # find derivative
    d1 = diff(series, b1)
    d2 = diff(series, b2)
    d3 = diff(series, b3)
    d = (d1, d2, d3)
    # find the volume lower bound
    vol_min_global = 1/h_max**3
    # divide solution space into grids
    x_max, y_max, z_max = b_max
    x = np.linspace(0, x_max, x_max+1)
    y = np.linspace(0, y_max, y_max+1)
    z = np.linspace(0, z_max, z_max+1)
    xv, yv, zv = np.meshgrid(x, y, z)
    for i in range(x_max):
        for j in range(y_max):
            for k in range(z_max):
                # find the bounds
                b1_min = xv[i,j,k]
                b1_max = b1_min+1
                b2_min = yv[i,j,k]
                b2_max = b2_min+1
                b3_min = zv[i,j,k]
                b3_max = b3_min+1
                bounds = [[b1_min, b1_max],[b2_min,b2_max],[b3_min,b3_max]]
                # try solve
                vol, sol = NSolve(series, d, vol_min_global, bounds)
                if type(sol) == int or type(vol) == int:
                    # sol = -1 and vol = -1
                    print ('range ', b1_min,b2_min,b3_min,' does not work')
                    continue
                if type(sol) == np.ndarray:
                    # edge case
                    sol = sol.tolist()
                if sol[0] > sol_max or sol[1] > sol_max or sol[2] > sol_max:
                    print ('solution out of bounds.')
                    continue
                if vol_min_global < vol < 1:
                    print ('vol:', vol, '; sol:', sol)
                    return vol, sol
    print('cannot find solution.')
    return -1,-1

def generate_vol(h_max, coeff=2):
    # Input:
    #   h_max: the max height of the polytope
    #   coeff: an empirical factor that determines the search space
    # Output:
    # triang_list: data for each polytope of height [h1,h2,h3]
    #   [h1,h2,h3]: heights
    #   vol: minimum volume
    #   sol: solution (i.e. the b's in the hilber series)
    #   prism: polytope point set
    #   series: hilbert series
    #   triang: triangulation of the polytope
    #   power: order of the t's in the hilbert series
    triang_list = []
    for h1 in range(1,h_max):
        for h2 in range(h1+1):
            for h3 in range(h2+1):
                print(h1,h2,h3)
                prism, series, triang, power = lift_prism(h1,h2,h3)
                b_max = (h1*coeff,h1*coeff,h1*coeff)
                vol, sol = solver(series, b_max, h1*coeff)
                triang_list.append([[h1,h2,h3], vol, sol, prism, series, triang, power])
                print('')
    return triang_list
    
print('Okay.')

Okay.


In [5]:
generate_vol(2)

(1, 0, 0)
reset starting point:  0.867792114606 0.911752932277 0.45554879871
solution:  [ 1.50000114  1.4999978   1.00000027]
guessed vol:  0.5925925925839692
Done.
('vol:', 0.5925925925839692, '; sol:', [1.5000011404230877, 1.4999977968369254, 1.0000002656749278])

(1, 1, 0)
reset starting point:  0.699611038725 0.746124233374 0.480540812785
solution:  [ 1.50002675  1.5000362   1.49998898]
guessed vol:  0.5925925929914229
Done.
('vol:', 0.5925925929914229, '; sol:', [1.5000267523925717, 1.5000361982110724, 1.4999889790500398])

(1, 1, 1)
reset starting point:  0.328846304563 0.152112459992 0.224381209018
solution:  [ 1.50000496  1.49999872  2.4999978 ]
guessed vol:  0.592592592627625
Done.
('vol:', 0.592592592627625, '; sol:', [1.5000049595595233, 1.499998721948516, 2.4999978015608537])



[[[1, 0, 0],
  0.5925925925839692,
  [1.5000011404230877, 1.4999977968369254, 1.0000002656749278],
  [[[1, 0, 0], [0, 0, 0], [0, 0, 1], [1, 1, 0]],
   [[0, 1, 0], [0, 0, 0], [0, 0, 1], [1, 1, 0]]],
  1/((b1 - b2)*b1*(b2 + b3 - 4)*b3) - 1/((b1 - b2)*(b1 + b3 - 4)*b2*b3),
  [array([[[1, 0, 0, 1],
           [0, 0, 0, 1],
           [0, 0, 1, 1],
           [1, 1, 0, 1]]]), array([[[0, 1, 0, 1],
           [0, 0, 0, 1],
           [0, 0, 1, 1],
           [1, 1, 0, 1]]])],
  [array([[[ 0.,  1.,  0.,  0.],
           [ 1., -1., -0., -0.],
           [-1.,  0., -1.,  1.],
           [-0., -0.,  1., -0.]]]), array([[[ 1., -0., -0., -0.],
           [-1.,  1.,  0.,  0.],
           [-0., -1., -1.,  1.],
           [ 0.,  0.,  1.,  0.]]])]],
 [[1, 1, 0],
  0.5925925929914229,
  [1.5000267523925717, 1.5000361982110724, 1.4999889790500398],
  [[[1, 0, 0], [0, 0, 0], [0, 0, 1], [1, 1, 0]],
   [[0, 1, 1], [0, 0, 0], [0, 0, 1], [1, 1, 0]]],
  1/((b1 - b2 + b3)*(b1 - b2)*(b1 + b3 - 4)*b1) - 1/((b1 -

In [22]:
h1 = 2
h2 = 1
h3 = 0
print(h1,h2,h3)
prism, series, triang, power = lift_prism(h1, h2, h3)
triang = np.squeeze(np.array(triang))
power = np.squeeze(np.array(power))
print(series)
print(prism)
print(triang)
print(power)

(2, 1, 0)
(0, 1, 2)
1
1/((2*b1 + b2 + b3 - 12)*(2*b1 + b3 - 8)*(b1 - b2)*b1) + 1/((b1 + b3 - 4)*b1*(b2 + b3 - 4)*(b2 - 4)) - 1/((3*b1 + b3 - 12)*(2*b1 + b3 - 8)*(b1 - b2)*b2) + 1/((b1 - b2)*b1*(b2 + b3 - 4)*b3) - 1/((b1 - b2)*(b1 + b3 - 4)*b2*b3)
[[[1, 0, 0], [1, 1, 0], [0, 0, 0], [0, 0, 1]], [[0, 0, 0], [1, 1, 0], [0, 1, 0], [0, 0, 1]], [[1, 1, 0], [0, 1, 0], [0, 1, 1], [0, 0, 1]], [[1, 0, 0], [1, 1, 0], [0, 0, 2], [0, 0, 3]], [[1, 1, 0], [0, 1, 2], [0, 0, 2], [0, 0, 3]]]
[[[1 0 0 1]
  [1 1 0 1]
  [0 0 0 1]
  [0 0 1 1]]

 [[0 0 0 1]
  [1 1 0 1]
  [0 1 0 1]
  [0 0 1 1]]

 [[1 1 0 1]
  [0 1 0 1]
  [0 1 1 1]
  [0 0 1 1]]

 [[1 0 0 1]
  [1 1 0 1]
  [0 0 2 1]
  [0 0 3 1]]

 [[1 1 0 1]
  [0 1 2 1]
  [0 0 2 1]
  [0 0 3 1]]]
[[[ 0.  0.  1.  0.]
  [ 1. -1. -0. -0.]
  [ 0.  1.  0.  0.]
  [-1. -0. -1.  1.]]

 [[ 0.  0.  1.  0.]
  [-0. -1. -1.  1.]
  [ 1.  0.  0.  0.]
  [-1.  1. -0. -0.]]

 [[-0. -1. -0.  1.]
  [ 1.  0.  0.  0.]
  [-1. -0. -1.  1.]
  [ 0.  1.  1. -1.]]

 [[ 2.  0.  1. -2.]
  [ 1.

In [25]:
data_file = open('lift_1_40.txt', 'r')
data_x = []
data_y = []
for line in data_file:
  data = eval(line)
  if data[1] > 0:
    data_x.append(data[0])
    data_y.append(1/data[1])
    
print ('Number of data: ', len(data_x))

('Number of data: ', 9506)


In [41]:
series_file = open('series_2.txt', 'w')
triang_file = open('triang.txt', 'w')
power_file = open('power.txt', 'w')
triang_file.close()
power_file.close()

triang_list = []
power_list = []

for data in data_x:
    h1 = data[0]
    h2 = data[1]
    h3 = data[2]
    prism, series, triang, power = lift_prism(h1, h2, h3)
    triang = np.squeeze(np.array(triang))
    power = np.squeeze(np.array(power))
    triang_list.append(triang)
    power_list.append(power)
    series_file.write("%s\n"%(str(series)))
series_file.close()

triang_list = np.squeeze(np.array(triang_list))
power_list = np.squeeze(np.array(power_list))
np.savetxt('triang.txt', triang_list, fmt='%d')
np.savetxt('power.txt', power_list, fmt='%d')

In [8]:
test_file = open('test_triang.npy', 'w')
test_file.close()
for i in range(1,10):
    test_file = open('test_triang.npy', 'a')
    a1 = np.zeros((i,2))
    a2 = np.ones((2,i))
    test_file.close()

NotImplementedError: Multiple enlargeable (0-)dimensions are not supported.

In [6]:
test_file = open('test_triang.npy', 'r')
fsz = os.fstat(test_file.fileno()).st_size

IOError: Failed to interpret file <open file 'test_triang.npy', mode 'r' at 0x7f8028466db0> as a pickle